# DLfsP CNN am Beispiel PSF
Kurs Deep Learning für sequentielle Prozessdaten

#### In diesem Notebook werden CNN Layer auf Zeitreihendaten einer Profilschienenführung (PSF) angewendet. 

Linearführungen und Profilschienenführungen sind für einen nicht unbedeutenden Teil der Ausfälle bei Werkzeugmaschinen.
Typische Fehlerfälle die dabei auftreten sind Mangelschmierungen, Pittings an Laufbahnen oder an den Wälzkörpern. In diesem Beispiel gibt es Daten von Normalfahrten und vom Zustand Pitting.  

Während der Versuche wurde mittels einem 3-achsigen MEMS-Sensor Daten vom Wagen aufgenommen. 
Für diese Aufgabe wurde nur die Beschleunigung in Verfahrrichtung (Acc_X) verwendet. Grundsätzlich handelt es sich um die gleichen Daten wie beim SimpleRNN, allerdings wurden nun längere Zeitreihen mit einer höheren Genauigkeit aufgenommen. Ziel ist die Erkennung, also Klassifikation nach dem Zustand ('Status'). 

### Data-Mining-Prozess:
![Daten erfassen -> Daten erkunden -> Daten vorbereiten -> Modelle bilden -> Modelle validieren -> Modell testen & anwenden](Prozess_Modellentwicklung_v2.png "model development")

## 0. Bibliotheken importieren

In [ ]:
''' Import der wichtigsten Pakete. '''
import numpy as np
import pandas as pd
import random 
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import seaborn as sns

#Einstellungen für die Grafikausgabe
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen


In [ ]:
''' Überprüfe, ob TensorFlow installiert ist und die GPU oder die CPU nutzt. '''
print('TF version: ' + tf.__version__)
if tf.test.is_built_with_gpu_support():
    if len(tf.config.list_physical_devices('GPU'))==0:
         print('Achtung, TensorFlow nutzt gerade die CPU und nicht die GPU!')
    else:
        print('Default GPU Device: {}'.format(tf.config.list_physical_devices('GPU')))
        # GPU-Memory-Management:
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.compat.v1.Session(config=config)
else:
    print('TensorFlow CPU version active')

TODO:
* Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen my_seed=

Damit Ergebnisse Reproduzierbar sind, müssen mehrere Seeds für Zufallszahlengeneratoren gesetzt werden. 

AUSGABE:
* Gewählte Zufallszahl

In [ ]:
''' Setzen von Seeds, um Reproduzierbarkeit zu ermöglichen. '''
# Erstelle eigene Zufallszahlen
my_seed = TODO

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

# Seeds für diverse Zufallszahlengeneratoren setzen 
os.environ['PYTHONHASHSEED']=str(my_seed)
tf.random.set_seed(my_seed)
random.seed(my_seed)
np.random.seed(my_seed)

## 1. Daten erfassen

In [ ]:
df = pd.read_csv('PSF_Pitting.csv') 
print('Daten erfolgreich importiert')

## 2. Daten erkunden

In [ ]:
""" ersten Datensätze als Beispiel"""
df.head(5)

In [ ]:
""" Statistische Beschreibung Datensatz"""
df.describe()

In [ ]:
# Einige Zyklen plotten
cycles=[]
cycles = random.sample(df['run_ID'].drop_duplicates().to_list(),3)

plt.figure(figsize=(12, 8))
plt.ylabel('Beschleunigung in X')
for c in cycles:
    status = df[df['run_ID']==c].reset_index().loc[0,'Status']
    plt.plot(df[df['run_ID']==c]['time_ms'],df[df['run_ID']==c]['Acc_X'],label='Run '+str(c) + ' Status ' + str(status))

plt.xlabel('Zeit in ms')
plt.legend()
plt.show()

## 3. Daten vorbereiten

In [ ]:
""" Aufteilung in X und y"""
# Maximale Zykluslänge
maxlen = df['run_ID'].value_counts().max()    

# Aufteilung X und y Daten als DataFrames
y_df = df[['Status','run_ID']]
X_df = df.reset_index(drop=True)

In [ ]:
""" Umwandeln der DataFrames in numpy arrays und group by run_ID"""
g = X_df.groupby('run_ID').cumcount()
X = (X_df.set_index(['run_ID',g])
     .unstack()
     .stack().groupby(level=0)
     .apply(lambda x: x['Acc_X'].values)
     .to_numpy())
g = y_df.groupby('run_ID').cumcount()
y = (y_df.set_index(['run_ID',g])
     .unstack()
     .stack().groupby(level=0)
     .apply(lambda x: np.rint(x.sum()/len(x)))
     .to_numpy().astype("int32"))

print('Shapes der numpy Arrays X und y:')
print(X.shape, y.shape)

In [ ]:
'''Auffüllen der ungleichen Messreihen'''
# Auffüllen mit führenden Nullen, damit die Zeitreihen die gleiche Länge haben
X_pre = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=maxlen, padding='pre', dtype="float32")

In [ ]:
""" Aufteilen in Trainings-, Validierungs und Testdaten. """
# Festlegen des Anteils an Test- und Validierungsdaten
test_split = 0.2
validation_split = 0.2

# Aufteilung der Daten
# Testdaten abspalten# Aufteilung der Daten
X_train, X_test, y_train, y_test = train_test_split(X_pre, y, 
                                                    test_size=test_split, 
                                                    shuffle=True, 
                                                    random_state=my_seed)

# Aufsplitten in Trainings- und Validierungsdaten
X_train, X_val, y_train ,y_val = train_test_split(X_train, y_train, 
                                                  test_size=validation_split, 
                                                  shuffle=True, 
                                                  random_state=my_seed)

nr_train=len(X_train)
nr_val=len(X_val)
nr_test=len(X_test)
print(nr_train, "Training sequences")
print(nr_val, "Validation sequences")
print(nr_test, "Test sequences")

In [ ]:
""" Daten normalisieren. """
# Importieren und Anlernen des Scalers (d.h. Scaler lernt Mittelwerte und Standardabweichungen der Features)
scaler = StandardScaler().fit(X_train)

# Normierung der Daten mit den gelernten Paramatern und gleichzeitiges Formatieren
X_train = scaler.transform(np.vstack(X_train)).reshape(nr_train, maxlen, -1)
X_val = scaler.transform(np.vstack(X_val)).reshape(nr_val, maxlen, -1)
X_test = scaler.transform(np.vstack(X_test)).reshape(nr_test, maxlen, -1)
print('Datenvorbereitung fertig')

In [ ]:
''' Überprüfen der Form der Numpy Arrays. '''
print('Shape X_train:\t' + str(X_train.shape))
print('Shape X_val:\t' + str(X_val.shape))
print('Shape X_test:\t' + str(X_test.shape))

## 4. Modelle bilden - pur CNN Modell

### 1.) Modellaufbau des CNN Modells

Zuerst wird ein reines CNN Modell erstellt. Dessen Performance wird danach mit einem mixed Modell aus CNN Layer und RNN Layern verglichen.

TODO:
* Importiere die notwendigen Layer
* Definiere eine Modellstruktur 
* Finde eine gute Anzahl der Layer und der Filter/Knoten je Layer
* Probiere also verschiedene Modellgrößen aus, um das beste Modell für die Aufgabe zu finden

In [ ]:
'''Modellaufbau'''
# Import der notwendigen Klassen und Funktionen
TODO

# Definition des Input_Shapes
input_shape = (TODO, TODO)

# Definition des Modells
model = Sequential()
"""
*
* TODO: 
*      Layers hinzufügen
*     
"""

model.summary()

### 2.) Modellkompilierung des CNN Modells
TODO: 
* Wähle einen Optimierer aus (Adam, Nadam, RMSprop oder SGD) 
* Lege - falls notwendig - die Parameter des Optimiers fest, Lernrate o.ä.
* Bestimme, welche Loss Funktion verwendet werden soll 

Hinweis:  
Die Lernrate ist ein sehr wichtiger Hyperparameter, der viel Einfluss auf den Verlauf und Schnelligkeit des Trainings hat!

In [ ]:
''' Modell kompilieren. '''
# Optimizer festlegen
optimizer = TODO

# Einstellung für das spätere Training zum Kompilieren festlegen
model.compile(optimizer=optimizer, loss=TODO, metrics=['acc'])

### 3.) Trainieren des CNN Modells

TODO:
* Wähle die Anzahl der Epochen  und die Batch_Size aus, für die das Modell trainiert werden soll. 
* Danach müssen die Trainingsinputs, -outputs, die batch_size, die Anzahl der Epochen sowie die Validierungsdaten bestimmt werden.

Ausgabe: 
* Zwischenergebnisse je Epoche

In [ ]:
''' Modell trainieren '''
# Festlegung der Batch_Size und Anzahl der Epoche
batch_size = TODO
epochs = TODO

# Durchführen des Trainings. Die Ergebnisse je Epoche werden in der History gespeichert. 
history_cnn = model.fit(TODO)

# Modell in extra Variable speichern
model_cnn = model

## 5. Modelle validieren - CNN Modell

### Kennzahlen fürs Training und Validierung:
TODO:
* Berechne den Loss und die Accuracy für das CNN Modell

Tipp:
* Nutze den evaluate Befehl mit den jeweils richtigen Daten

In [ ]:
print('Kennzahlen (Loss und Accuracy Metrik) fürs Training beim CNN Modell:')
print(model_cnn.TODO)
print('Kennzahlen (Loss und Accuracy Metrik) für Validierung beim CNN Modell:')
print(model_cnn.TODO)

## Visualisierung der Ergebnisse

In [ ]:
""" Trainingsverlauf plotten. """
train_acc = history_cnn.history['acc']
val_acc = history_cnn.history['val_acc']
train_loss = history_cnn.history['loss']
val_loss = history_cnn.history['val_loss']
epochs = range(1,len(train_acc) +1)

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_acc,'bo', label='Training accuracy')
plt.plot(epochs, val_acc,'b', label='Validation accuracy')
plt.xlim((0, max(epochs)))
plt.ylim(top=1.0)  # adjust the top leaving bottom unchanged
plt.title('Loss Dense NN', fontsize = 16)
plt.title('Accuracy', fontsize=16)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.xlim((0, max(epochs)))
plt.ylim(bottom=0.0)  
plt.title('Loss Dense NN', fontsize = 16)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## 4. Modelle bilden - CNN + RNN Modell
Nun soll ein kombiniertes Modell erstellt werden. Bei diesem Modell sollen die Daten erst mittel CNN Layern verdichtet werden bevor danach ein RNN Schicht, also LSTM oder GRU angewendet wird. Die Performance dieses Modells soll danach mit dem des reinen CNN Modells verglichen werden.

### 1. Modellaufbau des CNN + RNN Modells

TODO:
* Importiere die notwendigen Layer
* Definiere eine Modellstruktur bestehend aus Convolutional und RNN Schichten
* Finde eine gute Anzahl der Layer und der Filter/Knoten je Layer
* Probiere also verschiedene Modellgrößen aus, um das beste Modell für die Aufgabe zu finden


In [ ]:
''' Modellaufbau '''
# Import der notwendigen Layer
TODO

# Definition des Input_Shapes
input_shape = (TODO, TODO)


# Definition des Modells
model = Sequential()
"""
*
* TODO: 
*      Layers hinzufügen
*     
"""

model.summary()

### 2.) Modellkompilierung des CNN + RNN Modells
TODO: 
* Wähle einen Optimierer aus (Adam, Nadam, RMSprop oder SGD) 
* Lege - falls notwendig - die Parameter des Optimiers fest, Lernrate, GradientClipping o.ä.
* Bestimme, welche Loss Funktion verwendet werden soll

Hinweise:  
Die Lernrate ist ein sehr wichtiger Hyperparameter, der viel Einfluss auf den Verlauf und Schnelligkeit des Trainings hat!

In [ ]:
''' Modell kompilieren. '''
# Optimizer festlegen
optimizer = TODO

# Einstellung für das spätere Training zum Kompilieren festlegen
model.compile(optimizer=optimizer, loss=TODO, metrics=['acc'])

### 3.) Modelltraining des CNN + RNN Modells

TODO:
* Wähle die Anzahl der Epochen und die Batch_Size aus, für die das Modell trainiert werden soll. 
* Danach müssen die Trainingsinputs, -outputs, die batch_size, die Anzahl der Epochen sowie die Validierungsdaten bestimmt werden.

Ausgabe: 
* Zwischenergebnisse je Epoche

In [ ]:
''' Modell trainieren '''
# Festlegung der Batch_Size und Anzahl der Epoche
batch_size = TODO
epochs = TODO

# Durchführen des Trainings. Die Ergebnisse je Epoche werden in der History gespeichert. 
history_rnn = model.fit(TODO)

# Modell in extra Variable speichern
model_rnn = model

## 5. Modelle validieren - CNN + RNN Modell

### Kennzahlen fürs Training und Validierung:
TODO:
* Berechne den Loss und die Accuracy für das CNN + RNN Modell

Tipp:
* Nutze den evaluate Befehl mit den jeweils richtigen Daten

In [ ]:
''' Bewertung des Trainings anhand der Valididierungsdaten mittels der Accuracy. '''
print('Kennzahlen (Loss und Accuracy Metrik) fürs Training beim CNN ü Modell:')
print(model_rnn.TODO)
print('Kennzahlen (Loss und Accuracy Metrik) für Validierung beim GRU Modell:')
print(model_rnn.TODO)

## Visualisierung der Ergebnisse

In [ ]:
""" Trainingsverlauf plotten. """
train_acc = history_rnn.history['acc']
val_acc = history_rnn.history['val_acc']
train_loss = history_rnn.history['loss']
val_loss = history_rnn.history['val_loss']
epochs = range(1,len(train_acc) +1)

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_acc,'bo', label='Training accuracy')
plt.plot(epochs, val_acc,'b', label='Validation accuracy')
plt.xlim((0, max(epochs)))
plt.ylim(top=1.0)  # adjust the top leaving bottom unchanged
plt.title('Accuracy SimpleRNN', fontsize=16)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.xlim((0, max(epochs)))
plt.ylim(bottom=0.0)  
plt.title('Loss SimpleRNN', fontsize = 16)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## 5. Modelle validieren - Vergleich CNN und CNN + RNN
TODO:
* Berechne den Loss und die Accuracy für die beiden Modelle jeweils für Training und Validierung 

In [ ]:
print('Kennzahlen (Loss und Accuracy Metrik) fürs Training beim CNN Modell:')
print(model_cnn.TODO)
print('Kennzahlen (Loss und Accuracy Metrik) für Validierung beim CNN Modell:')
print(model_cnn.TODO)
print('Kennzahlen (Loss und Accuracy Metrik) fürs Training beim CNN + RNN Modell:')
print(model_rnn.TODO)
print('Kennzahlen (Loss und Accuracy Metrik) für Validierung beim CNN + RNN Modell:')
print(model_rnn.TODO)

### Modell auswählen
Wähle ein finales Modell, also model_cnn oder model_rnn, aus, dass nun getestet werden soll.   
Dieses Modell wird in der Variable model gespeichert, mit der im Schritt 6. weitergearbeite wird

In [ ]:
model = TODO

## 6. Modell testen & anwenden

Führen Sie diesen Schritt erst aus, wenn Sie mit Ihrem Modell zufrieden sind.  

Der Test eines Modells soll ermitteln, wie gut das Modell übertragbar und generalisierbar ist.  
Wenn alle Modelle direkt getestet, wird einfach nur das Modell ausgewählt, das am Besten zum Testdatensatz passt. Dadurch bekommt man aber nicht das Modell, das das Problem grundsätzlich am Besten löst. Es gibt somit ein Overfitting bzgl. den Testdaten.

TODO:
* Berechne den Loss und die Accuracy anhand der Testdaten

In [ ]:
''' Bewertung des Trainings anhand der Valididierungsdaten mittels der Accuracy. '''
print('Kennzahlen (Loss und Accuracy Metrik) bei Testen:')
model.TODO

In [ ]:
"""Modell anhand Testdaten bewerten"""
y_test_pred = (model.predict(X_test)>0.5).astype("int32")[:, -1, :]
# Berechne Genauigkeit auf den Testdaten
accuracy_test = accuracy_score(y_test, y_test_pred)

# Berechne den F1-Score auf den Testdaten
f1score_test = f1_score(y_test, y_test_pred)

# Ausgabe der Modellgenauigkeit
print('Ergebnis für den Test:')
print('Accuracy: \t' + str(round(accuracy_test, 4)))
print('F1-Score: \t' + str(round(f1score_test, 4)))

# Visualisierung der Konfusionsmatrix
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
plt.grid()
plt.title('Konfusionsmatrix auf Testdaten')
plt.show()

# Classification Report
print('Classification Report')
print(classification_report(y_test, y_test_pred))